In [5]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

In [ ]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

In [3]:
from tapas.utils import sqa_utils, interaction_utils

In [8]:
with tf.io.gfile.GFile('data/SQA/random-split-1-dev.tsv', 'r') as file_handle:
    questions = interaction_utils.read_from_tsv_file(file_handle)

In [ ]:
questions[0]

In [7]:
from importlib import reload 

In [9]:
import table
import intermediate_pretrain_synthetic
from intermediate_pretrain_synthetic import CFG

In [243]:
table = reload(table)
import table

i+=1
t = table.load_table(wnums[i])
t.df

,Service,2012/13 Total Cost (PSmillion),Comparison with 2011/12 (PSmillion)
0,BBC Radio 1,54.2,+ 3.6
1,BBC Radio 1Xtra,11.8,+ 0.7
2,BBC Radio 2,62.1,+ 1.6
3,BBC Radio 3,54.3,+ 1.8
4,BBC Radio 4,122.1,+ 6.2
5,BBC Radio 4 Extra,7.2,- 1
6,BBC Radio 5 Live,76.0,+ 6.7
7,BBC Radio 5 Live Sports Extra,5.6,+ 0.3
8,BBC Radio 6 Music,11.5,- 0.2
9,BBC Asian Network,13.0,0


In [251]:
intermediate_pretrain_synthetic = reload(intermediate_pretrain_synthetic)
from intermediate_pretrain_synthetic import CFG
cfg = CFG(True, t)
t_statement = cfg.get_statement()
cfg.result = False
f_statement = cfg.get_statement()
print(t_statement)
print(f_statement)

aggregate empty list
2012/13 Total Cost (PSmillion) when 2012/13 Total Cost (PSmillion) is less than 669.5 and 2012/13 Total Cost (PSmillion) is greater than 54.2 is less than 669.5
the count when 2012/13 Total Cost (PSmillion) is greater than 11.5 is 1


In [245]:
t.numeric_columns

{'2012/13 Total Cost (PSmillion)'}

In [168]:
import os
table = reload(table)
filenames = os.listdir('data/SQA/table_csv/')
wnums = []
count = 0
for name in filenames:
    t = table.load_table('data/SQA/table_csv/' + name)
    if len(t.numeric_columns) > 0:
        count += 1
        wnums.append('data/SQA/table_csv/' + name)

print(count, len(filenames))

626 982


In [154]:
t.df

,Season,Competition,Round,Club,Home,Away,Aggregate
0,2003,UEFA Intertoto Cup,R1,WIT Georgia,1-0,1-2,2-2
1,,,R2,FK Pobeda,2-1,1-1,3-2
2,,,R3,Tobol Kostanay,3-0,1-0,4-0
3,,,Semifinals,Werder Bremen,4-0,1-1,5-1
4,,,Finals,FC Schalke 04,0-2,0-0,0-2
5,2004/05,UEFA Cup,Q2,Zenit St. Petersburg,3-1,0-2,3-3
6,2005/06,UEFA Cup,Q2,Zenit St. Petersburg,2-2,1-1,3-3
7,2006/07,UEFA Cup,1,Livorno,0-1,0-2,0-3


In [89]:
iter(1)

TypeError: 'int' object is not iterable

In [175]:
import random
max([random.randint(0,2) for _ in range(10000)])

2

In [276]:
from tqdm import tqdm
import pandas as pd
import os
table = reload(table)
intermediate_pretrain_synthetic = reload(intermediate_pretrain_synthetic)
from intermediate_pretrain_synthetic import CFG

filenames = os.listdir('data/SQA/table_csv/')
data = ['name', 'result', 'statement']
for name in tqdm(filenames):
    t = table.load_table('data/SQA/table_csv/' + name)
    cfg = CFG(True, t)
    results = []
    for result in [True, True, False, False]:
        cfg.result = result
        statement = cfg.get_statement()
        if statement != None:
            if statement in results:
                statement = cfg.get_statement()
            if statement not in results:
                results.append((result, statement))

    if results:
        data.extend([[name, result, statement] for result, statement in results])

100%|██████████| 982/982 [00:27<00:00, 36.16it/s]


In [277]:
len(data)

3927

In [266]:
import numpy as np
a = np.array([1,1,1])
b = np.array([3,2,1])

(a == 1).all()

True

In [267]:
a = set([3,2,1])
b = set([1,2,3])

a == b

True

TypeError: 'int' object is not iterable

In [278]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

In [307]:
max_seq_length = 512
vocab_file = "data/models/tapas_masklm_medium_reset/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")

In [293]:
data[3]

['200_12.csv', True, 'the count when Award is Tony Award is 4']

In [294]:
from collections import defaultdict
synth_data = defaultdict(list)

for name, result, statement in data[3:]:
    synth_data[name].append((result, statement))

In [323]:
from tapas.utils import interaction_utils
interactions = []
for name in os.listdir('data/SQA/'):
    if 'random' in name and 'dev' in name:
        with open('data/SQA/' + name, 'r') as file:
            interactions.extend(interaction_utils.read_from_tsv_file(file))

In [314]:
interactions[0].table.table_id[10:]

'203_583.csv'

In [302]:
from tapas.utils import text_utils
from tapas.utils.synthesize_entablement import _from_table_proto, _to_table_proto

def synthesize_from_interaction(
    interaction,
    statements
):
  """Synthesizes up to for new statements for a given interaction.

  Args:
    config: Specifies sampling hyper-parameters.
    rng: A random number generator.
    interaction: Table will be used to generate random statements.
    counter: For aggregating statistics.
    add_opposite_table: If true, add opposite table.

  Returns:
    A list of up to 4 statements.

  Attempts to synthesize one positive and one negative statement.
  For each statement attempts to generate a table with the opposite outcome.
  """
  table = _from_table_proto(interaction.table)

  def get_interaction(interaction, table, statement, result, name):
    new_interaction = interaction_pb2.Interaction()
    if interaction.id:
      new_interaction.id = interaction.id
    else:
      new_interaction.id = interaction.table.table_id
    new_interaction.table.CopyFrom(_to_table_proto(table))
    new_interaction.table.table_id = interaction.table.table_id
    new_question = new_interaction.questions.add()
    new_question.id = new_interaction.id + '_' + name
    new_question.original_text = statement
    new_question.text = text_utils.normalize_for_match(
        new_question.original_text)
    if result == True:
      new_question.answer.class_index = 1
    elif result == False:
      new_question.answer.class_index = 0
    else:
      raise ValueError('Unexpected: {result}')
    return new_interaction

  interactions = []

  for i, (result, statement) in enumerate(statements):
      interactions.append(
          get_interaction(interaction, table, statement,
                          result, '{}{}'.format(result, i)))
      

  return interactions

In [324]:
synth_interactions = []

In [326]:
synth_interactions = []
for interaction in tqdm(interactions):
    name = interaction.table.table_id[10:]
    synth_interactions.extend(synthesize_from_interaction(interaction, synth_data[name]))

100%|██████████| 3891/3891 [00:00<00:00, 18489.52it/s]


In [325]:
len(synth_interactions)

0

In [327]:
converted_interactions = []
for interaction in tqdm(synth_interactions):
    for i in range(len(interaction.questions)):
      try:
        converted_interactions.append(converter.convert(interaction, i))
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}") 

100%|██████████| 15534/15534 [00:37<00:00, 413.41it/s]


In [310]:
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

In [328]:
import tensorflow.compat.v1 as tf
write_tf_example('data/intermediate_data/dev.tfrecord', converted_interactions)